In [19]:
import heapq

class WeightedGraph:
    def __init__(self, nodes):
        self.graph = {}
        self.weights = {}
        for node in range(nodes):
            self.graph[node] = []
            self.weights[node] = {}

    def has_edge(self, src, dst):
        return dst in self.graph[src]

    def add_edge(self, src, dst, weight):
        if not self.has_edge(src, dst):
            self.graph[src].append(dst)
            self.graph[dst].append(src)
            self.weights[(src, dst)] = weight
            self.weights[(dst, src)] = weight

    def get_graph(self):
        return self.graph

    def get_size(self):
        return len(self.graph)  # Corrected method to get the number of nodes

def initialize_distances_and_paths(graph, source):
    distances = {}
    paths = {}
    for node in graph.graph:
        distances[node] = float('inf')
        paths[node] = []
    distances[source] = 0
    paths[source] = [source]
    return distances, paths

def initialize_relax_count(graph):
    relax_count = {}
    for node in graph.graph:
        relax_count[node] = 0
    return relax_count

### Part 1.1

In [20]:
def dijkstra_k_relaxations(g, s, k):
    distances, paths = initialize_distances_and_paths(g, s)
    relax_count = initialize_relax_count(g)
    
    min_heap = [(0, s)]

    while min_heap:
        current_distance, current_node = heapq.heappop(min_heap)

        if relax_count[current_node] >= k:
            continue

        relax_count[current_node] += 1

        for neighbor in g.graph[current_node]:
            new_distance = current_distance + g.weights[(current_node, neighbor)]
            if new_distance < distances[neighbor]:
                distances[neighbor] = new_distance
                paths[neighbor] = paths[current_node] + [neighbor]
                heapq.heappush(min_heap, (new_distance, neighbor))

    return {'distance': distances, 'path': paths}

In [21]:
graph = WeightedGraph(5)
graph.add_edge(0, 1, 4)
graph.add_edge(0, 2, 2)
graph.add_edge(1, 2, 3)
graph.add_edge(1, 3, 1)
graph.add_edge(2, 4, 5)
graph.add_edge(3, 4, 7)

result = dijkstra_k_relaxations(graph, 0, 2)  # Find shortest paths with at most 2 relaxations per node
print(result)

{'distance': {0: 0, 1: 4, 2: 2, 3: 5, 4: 7}, 'path': {0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 1, 3], 4: [0, 2, 4]}}


### Part 1.2

In [22]:
def bellman_ford_k_relaxations(g, s, k):
    distances, paths = initialize_distances_and_paths(g, s)
    relax_count = initialize_relax_count(g)

    for _ in range(g.get_size() - 1):
        for u in g.graph:
            if relax_count[u] >= k:
                continue
            for v in g.graph[u]:
                new_distance = distances[u] + g.weights[(u, v)]
                if new_distance < distances[v]:
                    distances[v] = new_distance
                    paths[v] = paths[u] + [v]
                    relax_count[v] += 1

        for u in g.graph:
            for v in g.graph[u]:
                if distances[v] > distances[u] + g.weights[(u, v)]:
                    return None

    return {'distance': distances, 'path': paths}




In [23]:
# Example usage
graph = WeightedGraph(5)
graph.add_edge(0, 1, 4)
graph.add_edge(0, 2, 2)
graph.add_edge(1, 2, 3)
graph.add_edge(1, 3, 1)
graph.add_edge(2, 4, 5)
graph.add_edge(3, 4, -7)  # Introduce a negative cycle

result = bellman_ford_k_relaxations(graph, 0, 2)  # Find shortest paths with at most 2 relaxations per node
print(result)  # Output: None (negative cycle detected)

graph = WeightedGraph(5)
graph.add_edge(0, 1, 4)
graph.add_edge(0, 2, 2)
graph.add_edge(1, 2, 3)
graph.add_edge(1, 3, 1)
graph.add_edge(2, 4, 5)
graph.add_edge(3, 4, 7)

result = bellman_ford_k_relaxations(graph, 0, 3)
print(result)

None
{'distance': {0: 0, 1: 4, 2: 2, 3: 5, 4: 7}, 'path': {0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 1, 3], 4: [0, 2, 4]}}
